In [6]:
!pip3 install torch
!pip3 install tqdm
!pip3 install nltk
!pip3 install pandas
!pip3 install numpy
!pip3 install seaborn
!apt-get install wget
!pip3 install youtokentome

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
     |████████████████████████████████| 1.7MB 15.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 20.1MB/s eta 0:00:01


In [4]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm.auto import tqdm

import pandas as pd

import numpy as np

from torch.utils.data import Dataset, DataLoader

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
import math
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [8]:
#ИЗМЕНИТЬ ПУТЬ В ЗАВИСИМОСТИ ОТ ТОГО ГДЕ ЗАПУСКАЕМ КОД
try:
    df = pd.read_csv("ctx_quest.csv")
except:
    df = pd.read_csv("/Users/lilyakhoang/input/question_generation/ctx_quest.csv")

df.head()

,Unnamed: 0,context,question
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?
1,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...
2,2,"Managed by her father, Mathew Knowles, the gro...",When did Beyonce leave Destiny's Child and bec...
3,3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?
4,4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?


In [9]:
df_nonan = df.dropna()
df_nonan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86610 entries, 0 to 86820
Data columns (total 3 columns):
Unnamed: 0    86610 non-null int64
context       86610 non-null object
question      86610 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB


In [13]:
# для "обучения" bpe модели нам нужно сохранить данные для обучения в отдельный файл
# где будут построчно храниться тексты
err = 0
try:
    f = open('/Users/lilyakhoang/input/for_bpe_ctx_quest.txt', 'w')
    bpe_model_address = '/Users/lilyakhoang/input/for_bpe_ctx_quest.txt'
except:
    f = open('for_bpe_ctx_quest.txt', 'w')
    bpe_model_address = 'for_bpe_ctx_quest.txt'
for que in df_nonan.context:
    try:
        f.write(que + '\n')
    except:
        err += 1
for que in df_nonan.question:
    try:
        f.write(que + '\n')
    except:
        err += 1
f.close()
print(err)

0


In [16]:
# обучаем
vocab_size = 16000
model_path = 'bpe.model'

yttm.BPE.train(data=bpe_model_address,vocab_size=vocab_size, model=model_path)

In [17]:
tokenizer = yttm.BPE(model=model_path)

In [18]:
# давайте токенизируем наш датасет
# токенизирую батчами, потому что так быстрее
# также в начало добавляем токен bos (begin of sentence)

tokenized_ctx = []
tokenized_quest = []
batch_size = 256

for i_batch in tqdm(range(math.ceil(len(df_nonan.context) / batch_size))):
    
    tokenized_ctx.extend(tokenizer.encode(list(df_nonan.context[i_batch*batch_size:(i_batch+1)*batch_size]), bos=True))

for i_batch in tqdm(range(math.ceil(len(df_nonan.question) / batch_size))):

    tokenized_quest.extend(tokenizer.encode(list(df_nonan.question[i_batch*batch_size:(i_batch+1)*batch_size]), bos = True, eos=True))



In [19]:
tokenizer.vocab()[:15]

['<PAD>',
 '<UNK>',
 '<BOS>',
 '<EOS>',
 '▁',
 'e',
 't',
 'a',
 'i',
 'n',
 'o',
 'r',
 's',
 'h',
 'l']

In [20]:
class WordData(torch.utils.data.Dataset):
    def __init__(self, context_list, questions_list, context_len, questions_len, pad_index, eos_index):
        self.context_list = context_list
        self.questions_list = questions_list
        
        self.context_len = context_len
        self.questions_len = questions_len
        
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.context_list)
    
    def __getitem__(self, index):
        
        context = self.context_list[index][:self.context_len]
        pads_ctx = [self.pad_index] * (self.context_len - len(context))
#         print(len(pads_ctx))
        context = torch.tensor(context + pads_ctx).long()
        
        question = self.questions_list[index][:self.questions_len]
        pads_quest = [self.pad_index] * (self.questions_len - len(question))
        question = torch.tensor(question + pads_quest).long()
        
        return context, question

In [21]:
batch_size = 64

context_len = 80
quest_len = 20

pad_index = 0
eos_index = 3

In [22]:
validation_start_index = int(len(tokenized_ctx) * 0.05)

In [13]:
subset_size = 300
small_ex_ctx = tokenized_ctx[:subset_size]
small_ex_quest = tokenized_quest[:subset_size]
small_validation_start_index = int(len(small_ex_ctx) *0.05)

small_train_dataset = WordData(context_list=small_ex_ctx[:-small_validation_start_index], questions_list = small_ex_quest[:-small_validation_start_index],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

small_validation_dataset = WordData(context_list=small_ex_ctx[-small_validation_start_index:],questions_list = small_ex_quest[-small_validation_start_index:],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

len(small_train_dataset), len(small_validation_dataset)

(285, 15)

In [40]:
train_dataset = WordData(context_list=tokenized_ctx[:-validation_start_index], questions_list = tokenized_quest[:-validation_start_index],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

validation_dataset = WordData(context_list=tokenized_ctx[-validation_start_index:],questions_list = tokenized_quest[-validation_start_index:],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(82280, 4330)

In [41]:
train_loader = torch.utils.data.DataLoader(small_train_dataset, batch_size=64)
validation_loader = torch.utils.data.DataLoader(small_validation_dataset, batch_size=64)

In [42]:
len(train_loader)

1

In [47]:
train_dataset = WordData(context_list=tokenized_ctx[:-validation_start_index], questions_list = tokenized_quest[:-validation_start_index],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

validation_dataset = WordData(context_list=tokenized_ctx[-validation_start_index:],questions_list = tokenized_quest[-validation_start_index:],
                         context_len=context_len, questions_len = quest_len, pad_index=pad_index, eos_index=eos_index)

len(train_dataset), len(validation_dataset)

(82280, 4330)

In [48]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)

In [43]:
class EncoderRNN_inside_class(nn.Module):
    def __init__(self, hidden_size, embedding_size, embedding):#input_size
        super(EncoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size, batch_first = True)

    def forward(self, input, hidden, debug = False):
        if debug == True: 
            print("====ENCODING_FORWARD====")
            print("input.shape", input.shape)
#         embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.embedding(input)
        output = embedded
        if debug == True: 
            print("embedded/output.shape",embedded.shape,"hidden.shape", hidden.shape  )
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [44]:
class DecoderRNN_inside_class(nn.Module):
    
    def __init__(self, embedding, embedding_size,
                 hidden_size, output_size):
        super(DecoderRNN_inside_class, self).__init__()
        self.hidden_size = hidden_size
        self.vectors = embedding
        self.embedding = embedding
        self.gru = nn.GRU(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, debug = False):
        if debug == True:
          print("===FORWARD_DECODER===")
          print("input.shape {}, hidden.shape {}".format(input.shape,hidden.shape ))
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [45]:
class My_Seq2Seq(nn.Module):
    def __init__(self, embedding_size, hidden_size, vocab_size, 
                 device, pad_idx, eos_idx, sos_idx):
        super(My_Seq2Seq, self).__init__()
        
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        # Encoder network
        self.encoder = EncoderRNN_inside_class(hidden_size, 
                               embedding_size, 
                               self.embedding)
        
        # Decoder network        
        self.decoder = DecoderRNN_inside_class(self.embedding,
                               embedding_size,
                              hidden_size,
                              vocab_size)
        
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.sos_idx = sos_idx
        self.device = device
    def forward(self, input_sequence, output_sequence, debug = False):
        input_tokens = input_sequence[0]
        input_lengths = input_sequence[1]
        
        encoder_hidden = self.encoder.initHidden(len(input_sequence))
        encoder_output, encoder_hidden = self.encoder(input_sequence, encoder_hidden)
        batch_size = len(input_sequence)
        outputs = torch.zeros(batch_size, 20, self.vocab_size).to(self.device)
#         print(outputs.shape)
        for batch_element_index in range(batch_size):
            target_tensor = output_sequence[batch_element_index, :]
            if debug == True: 
                print("\n Start handling new element")
                print("target_tensor.shape {}, target_tensor {}".format(target_tensor.shape, target_tensor))

            decoder_input = torch.tensor([[self.sos_idx]], device=device)
            decoder_hidden = encoder_hidden[:,batch_element_index,:].unsqueeze(1)

            if debug == True: 
                print("go_to_decoder_with decoder_input {}, decoder_hidden.shape {}".format(decoder_input, decoder_hidden.shape))
            
            # Teacher forcing: Feed the target as the next input
            for di in range(20):
                if debug == True: 
                    print("iterate_over_target {}th time, decoder_input.shape {}".format(di, decoder_input.shape))
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                outputs[batch_element_index, di, :] = decoder_output
                decoder_input = torch.tensor([target_tensor[di]], device=device).unsqueeze(1)
                if debug == True: 
                    print("AFTER_DECODER_STEP decoder_output.shape {}, decoder_input.shape {}, decoder_input {}".format(decoder_output.shape, decoder_input.shape, decoder_input))
                if decoder_input[0][0] == 0:
                    break
        return outputs

In [37]:
def evaluate (model, iterator):
    epoch_loss = 0
    for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):
        with torch.no_grad():
            input_seq = torch.tensor(batch[0]).to(device)
            target_tokens = torch.tensor(batch[1]).to(device)
            output = model(input_seq, target_tokens)
            output = output.view(-1, output.size(-1))
            target_tokens = target_tokens.view(-1)
#             print("AFTER RESHAPE PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
            loss = criterion(output, target_tokens)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [33]:
def train(model, iterator, criterion, optimizer,epoch_number, debug = False):
   # Put the model in training mode!
    model.train()
    losses_list = []
    epoch_loss = 0
    progress_bar = tqdm(enumerate(iterator), total=len(iterator), desc='Epoch {}'.format(epoch_number + 1))
    for idx, batch in progress_bar:
        optimizer.zero_grad()
        input_seq = torch.tensor(batch[0]).to(device)
        target_tokens = torch.tensor(batch[1]).to(device)
#         print(len(batch), batch[0][0],batch[1][0])
        output = model(input_seq, target_tokens)
        if debug == True:
            print("PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
#         output = output.view(-1)
        output = output.view(-1, output.size(-1))
        target_tokens = target_tokens.view(-1)
        if debug == True:
            print("AFTER RESHAPE PREDICTED_OUTPUT {}, REAL_OUTPUT {}".format(output.shape, target_tokens.shape))
        loss = criterion(output, target_tokens)
        losses_list.append(float(loss))
        progress_bar.set_postfix(train_loss = np.mean(losses_list[-100:]))
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
#         break
    return epoch_loss / len(iterator)

In [ ]:
# MY VERSIOB
pad_idx = tokenizer.vocab().index("<PAD>")
eos_idx = tokenizer.vocab().index("<EOS>")
sos_idx = tokenizer.vocab().index("<BOS>")
# Size of embedding_dim should match the dim of pre-trained word embeddings!
embedding_dim = 300
hidden_dim = 512
vocab_size = len(tokenizer.vocab())
model = My_Seq2Seq(embedding_dim,
                 hidden_dim, 
                 vocab_size, 
                 device, pad_idx, eos_idx, sos_idx).to(device)
optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [ ]:


validation_losses =[]
train_losses = []

N_EPOCHS = 20
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, criterion, optimizer, epoch)
    train_losses.append(train_loss)
    
    if min(train_losses) == train_loss and len(train_losses) > 1:
        torch.save(model.state_dict, "best_seq2seq_no_att_no_early_stop")
        torch.save(optimizer.state_dict, "best_Adam_state_dict_no_att_no_early_stop")
    
    torch.save(model.state_dict, "last_seq2seq_no_att_no_early_stop")
    torch.save(optimizer.state_dict, "Adam_state_dict_no_att_no_early_stop")
    
#     #early stopping
#     test_loss = evaluate(model, validation_loader)
#     validation_losses.append(test_loss)
    
#     if len(validation_losses) > 1 and validation_losses[epoch] > validation_losses[epoch-1]:
#         print("stop")
#         break



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
